<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/spark_sql_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
# create SparkSession
spark = SparkSession\
.builder\
.master("local")\
.appName("SQP n Pyspark")\
.getOrCreate()

In [14]:
# load the files into the spark session
moviesdf=spark.read.option("Header","true").option("inferschema","true").format("csv").load("/content/movies.csv")
ratingsdf=spark.read.option("Header","true").option("inferschema","true").format("csv").load("/content/ratings.csv")

In [13]:
# we get the schrma with original data type vales
moviesdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [15]:
ratingsdf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [16]:
ratingsdf.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



In [18]:
ratingsdf.select("userID","rating").show(5)

+------+------+
|userID|rating|
+------+------+
|     1|   3.5|
|     1|   3.5|
|     1|   3.5|
|     1|   3.5|
|     1|   3.5|
+------+------+
only showing top 5 rows



In [19]:
ratingsdf.filter("rating>3.35").show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      2|   3.5|1112486027|
|     1|     29|   3.5|1112484676|
|     1|     32|   3.5|1112484819|
|     1|     47|   3.5|1112484727|
|     1|     50|   3.5|1112484580|
+------+-------+------+----------+
only showing top 5 rows



**Display the rating with count**

In [30]:
ratingsdf.groupBy("rating").count().orderBy(col("rating").desc()).show(10)

+------+-------+
|rating|  count|
+------+-------+
|   5.0| 981369|
|   4.5| 520181|
|   4.0|1884182|
|   3.5| 752323|
|   3.0|1457862|
|   2.5| 304124|
|   2.0| 486137|
|   1.5|  93622|
|   1.0| 231165|
|   0.5|  82327|
+------+-------+



In [31]:
# create or replace temp view
msql=moviesdf.createOrReplaceTempView("moviesdf")
rsql=ratingsdf.createOrReplaceTempView("ratingsdf")

In [33]:
spark.sql("select * from moviesdf limit 3").show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+



In [36]:
spark.sql("select * from moviesdf order by movieId desc limit 2").show()

+-------+-------------------+--------------------+
|movieId|              title|              genres|
+-------+-------------------+--------------------+
| 131262|   Innocence (2014)|Adventure|Fantasy...|
| 131260|Rentun Ruusu (2001)|  (no genres listed)|
+-------+-------------------+--------------------+



Movie name and rating count

In [37]:
moviesdf.describe().show()

+-------+-----------------+--------------------+------------------+
|summary|          movieId|               title|            genres|
+-------+-----------------+--------------------+------------------+
|  count|            27278|               27278|             27278|
|   mean|59855.48057042305|                NULL|              NULL|
| stddev|44429.31469707313|                NULL|              NULL|
|    min|                1|"""Great Performa...|(no genres listed)|
|    max|           131262|       貞子3D (2012)|           Western|
+-------+-----------------+--------------------+------------------+



In [38]:
ratingsdf.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|           6793292|           6793292|           6793292|             6793292|
|   mean| 23230.64994085342| 9084.226516098528|3.5235419734644116|1.1017641420781667E9|
| stddev|13523.147886505403|19822.287266354764|  1.05199925965389|1.6244475461904848E8|
|    min|                 1|                 1|               0.5|           789652004|
|    max|             46743|            131258|               5.0|          1427781797|
+-------+------------------+------------------+------------------+--------------------+



In [46]:
spark.sql('''select m.title,count(*) as No_of_rating from moviesdf m
join ratingsdf r on m.movieId=r.userId group by m.title order by No_of_rating desc limit 5''').show()

+--------------------+------------+
|               title|No_of_rating|
+--------------------+------------+
|Hour of the Wolf ...|        7515|
|It's All About Lo...|        4524|
|Millennium Mambo ...|        3145|
|  Crime Spree (2003)|        3014|
|Prince of Central...|        2711|
+--------------------+------------+

